In [3]:
import pandas as pd
import numpy as np
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go

# Load the Brent oil price data and events data
df = pd.read_csv('BrentOilPrices.csv', parse_dates=['Date'])
events_df = pd.read_csv('BrentOilEvents.csv', parse_dates=['Date'])
df.sort_values('Date', inplace=True)
events_df.sort_values('Date', inplace=True)

# Merge events with price data to get price for event markers (next trading day if event falls on non-trading day)
events_df = pd.merge_asof(events_df, df[['Date', 'Price']], on='Date', direction='forward')

# Prepare global log returns series for plotting
df.set_index('Date', inplace=True)
price_series = df['Price']
log_returns = np.log(price_series).diff()

# Initialize Dash app
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Brent Oil Price Dashboard"),
    dcc.DatePickerRange(
        id='date-range',
        start_date=df.index.min(),
        end_date=df.index.max(),
        display_format='Y-MM-DD'
    ),
    dcc.Dropdown(
        id='event-select',
        options=[{'label': ev, 'value': ev} for ev in events_df['Event']],
        value=list(events_df['Event']),  # default select all events
        multi=True,
        placeholder="Select events to display"
    ),
    dcc.Graph(id='price-chart'),
    dcc.Graph(id='returns-chart')
])

# Callback to update charts based on selected date range and events
@app.callback(
    Output('price-chart', 'figure'),
    Output('returns-chart', 'figure'),
    Input('date-range', 'start_date'),
    Input('date-range', 'end_date'),
    Input('event-select', 'value')
)
def update_charts(start_date, end_date, selected_events):
    # Filter price data by date range
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]
    # Filter events by date range and selection
    filtered_events = events_df[(events_df['Date'] >= start_date) & (events_df['Date'] <= end_date)]
    if selected_events:
        filtered_events = filtered_events[filtered_events['Event'].isin(selected_events)]
    # Create price chart with event markers
    fig_price = go.Figure()
    fig_price.add_trace(go.Scatter(x=filtered_df.index, y=filtered_df['Price'], mode='lines', name='Price'))
    if not filtered_events.empty:
        fig_price.add_trace(go.Scatter(
            x=filtered_events['Date'], y=filtered_events['Price'],
            mode='markers', marker_symbol='x', marker_color='red',
            name='Events', text=filtered_events['Event'],
            hovertemplate="%{x|%Y-%m-%d}: %{text}<extra></extra>"
        ))
    fig_price.update_layout(title='Brent Crude Oil Price', xaxis_title='Date', yaxis_title='Price (USD/barrel)', hovermode='x')
    # Create log returns chart
    filtered_returns = log_returns[(log_returns.index >= start_date) & (log_returns.index <= end_date)]
    fig_returns = go.Figure()
    fig_returns.add_trace(go.Scatter(x=filtered_returns.index, y=filtered_returns.values, mode='lines', name='Log Return'))
    fig_returns.update_layout(title='Brent Oil Log Returns', xaxis_title='Date', yaxis_title='Log Return', hovermode='x')
    return fig_price, fig_returns

# Run the app (if running as a script)
if __name__ == '__main__':
    app.run_server(debug=True)

ModuleNotFoundError: No module named 'dash'

In [4]:
pip install dash

Defaulting to user installation because normal site-packages is not writeable
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
   ---------------------------------------- 7.9/7.9 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 103.3/103.3 kB 6.2 MB/s eta 0:00:00
   --------------------------------------- 224.5/224.5 kB 13.4 MB/s eta 0:00:00
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.0.4
    Uninstalling Werkzeug-3.0.4:
      Successfully uninstalled Werkzeug-3.0.4
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
